<h2>Transfer Learning 

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow.keras.layers as tfl


from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (160, 160)
train_directory = '../datasets/tiny-imagenet-200/train'
validation_directory = '../datasets/tiny-imagenet-200/val'
train_dataset = image_dataset_from_directory(train_directory, 
                                            shuffle=True, 
                                            batch_size=BATCH_SIZE, 
                                            image_size=IMG_SIZE, 
                                            validation_split=0.2,
                                            subset='training',
                                            seed=42)
validation_dataset = image_dataset_from_directory(validation_directory,
                                            shuffle=True,
                                            image_size=IMG_SIZE, 
                                            batch_size=BATCH_SIZE,
                                            validation_split=0.2,
                                            subset='validation',
                                            seed=42)

In [ ]:
class_names = train_dataset.class_names

plt.figure(figsize=(7, 7))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])
        plt.axis('off')

<h4>Prefetch</h4>

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

<h4>Augment Data</h4>

In [ ]:
def data_augmenter():
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(RandomFlip('horizontal'))
    data_augmentation.add(RandomRotation(0.2))
    return data_augmentation

augmenter = data_augmenter()

for image, _ in train_dataset.take(1):
    plt.figure(figsize=(7, 7))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        augmented_image = augmenter(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0]/255)
        plt.axis('off')

<h3></h3>

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

IMG_SHAPE = IMG_SIZE + (3, )
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=True, weights='imagenet')

In [ ]:
base_model.summary()

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
base_model.trainable = False
image_var = tf.Variable(image_batch)
pred = base_model(image_var)

tf.keras.applications.mobilenet_v2.decode_predictions(pred.numpy(), top=2)

<h3>Freeze all layers of MobileNetV2 except the Top layer (FC Dense layer) and train it</h3>
<p>First we build the model, define the shapes of the input data and layers<br>
Then we compile the model, define the hyperparameters such as learning rate, type of loss, metrics etc.<br>
Then we <code>.fit</code> (train) the model, data is fed in the model in this stage and epochs is defined</p>

In [ ]:
def alpaca_model(image_shape=IMG_SIZE, data_augmentation=data_augmenter()):
    # Make space for 3 color channels i.e RGB therefore adding another dimension of value 3
    input_shape = image_shape + (3,)
    # Define the model, make sure to exclude the top most layer so we can add our own layer and train that particular layer
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    # Freeze the whole model
    base_model.trainable = False
    # I guess this is where we inject the input data, injection point is tf.keras.Input as far as I understand
    inputs = tf.keras.Input(shape=input_shape)
    # augment the input data to make it 9 times
    x = data_augmentation(inputs)
    # Don't know what this does
    x = preprocess_input(x)
    
    # This training parameter is for batch norm layer, I dont understand how 
    # This is functional api so we can pass parameters like this 
    x = base_model(x, training=False)
    # From here on out we are adding our own layers
    x = tfl.GlobalAveragePooling2D()(x)
    # Dropout to reduce overfitting
    x = tfl.Dropout(0.2)(x)

    # This would be the last layer, also this is the layer which we will train
    prediction_layer = tfl.Dense(1)
    outputs = prediction_layer(x)
    
    # As far as I understand, keras.Model take all the layers between inputs and outputs (both inclusive) 
    # and makes a model out of it
    model = tf.keras.Model(inputs, outputs)

    return model

In [ ]:
model2 = alpaca_model(IMG_SIZE, data_augmentation=data_augmenter())

<h4>Compile the model</h4>

In [ ]:
base_learning_rate = 0.01
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate), 
               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
               metrics=['accuracy'])

<h4>Train the model</h4>

In [13]:
initial_epochs = 5
history = model2.fit(train_dataset, validation_data=validation_dataset, epochs=initial_epochs)

Epoch 1/5
2500/2500 [==============================] - 60s 24ms/step - loss: -1117308.8750 - accuracy: 0.0049 - val_loss: 17837.3281 - val_accuracy: 0.0000e+00
Epoch 2/5
2500/2500 [==============================] - 60s 24ms/step - loss: -2599142.2500 - accuracy: 0.0049 - val_loss: 32056.1855 - val_accuracy: 0.0000e+00
Epoch 3/5
2500/2500 [==============================] - 59s 23ms/step - loss: -4082341.0000 - accuracy: 0.0049 - val_loss: 46275.6641 - val_accuracy: 0.0000e+00
Epoch 4/5
2500/2500 [==============================] - 59s 24ms/step - loss: -5564080.5000 - accuracy: 0.0049 - val_loss: 60496.6523 - val_accuracy: 0.0000e+00
Epoch 5/5
2500/2500 [==============================] - 60s 24ms/step - loss: -7042822.0000 - accuracy: 0.0049 - val_loss: 74702.1094 - val_accuracy: 0.0000e+00
